# TMDB API로 데이터 수집

## TMDB 영화 데이터 추가 수집

1. 대상: TMDB 영화 데이터
2. 기간: 2005년 1월 1일 ~ 2025년 11월 30일까지 (21년치)
3. 범위: 전체 데이터 중 vote_count >= 30, imdb_id가 있는 movie_id 기준으로 수집
4. 조건: 누락된 페이지가 0%여야 함, TMDB API 초당 요청 수(40 req/s) 제한 대응, 500페이지 제한 대응, 중단/재개 지원, 수집 파일은 하나의 Parquet로 저장, Jupyter에서 그대로 실행 가능한 코드, 5~6시간 내 수집 완료
5. 필수 컬럼 (순서대로): 19개 컬럼
- 공통 필수 컬럼
imdb_id
movie_id
vote_count
title

(1) 감독(directors) 전용 필수 컬럼
director_ids     = "525; 6193"
directors       = "Christopher Nolan; Denis Villeneuve"
director_gender    = "2; 2"
director_profile_path

(2) 작가(writer) 분석용 필수 컬럼
writer_ids    = "12452; 142521"
writers      = "Jonathan Nolan; Lisa Joy"
writer_roles  = job 기준 역할
writer_gender   = "2; 1"
writer_profile_path

(3) 주연 배우(top_cast) 상위 5명 전용 필수 컬럼
top_cast_ids   = "1190668; 505710; 61512"
top_cast_order  = "0; 1; 2; 3; 4"
top_cast     = "Timothée Chalamet; Zendaya; Rebecca Ferguson"
character (배역명)
top_cast_gender  = "2; 1; 1"
top_cast_profile_path

6. 주의사항:
- crew에는 감독/작가 외 다양한 직군(department/job)이 포함되므로, 필터 정확히 해야 함
- 감독: crew 안에서 job == "Director", 공동 감독 있을 수 있음 (대소문자 구분)
- 작가: writers = [
    c for c in crew_list
    if c.get("job") in ("Writer", "Screenplay", "Story", "Creator", "Novel", "Comic Book")
    or c.get("department") == "Writing"
  ]
 -> 그래야 원안, 각본, 집필자 모두 커버됨, 공동 작가 있을 수 있음 (대소문자 구분)
- 주연 배우 top 5: 
filtered = [c for c in sorted_cast if c.get("character")]
top_5 = filtered[:5]
-> cast 배열을 order 필드를 기준으로 오름차순 정렬 (반드시 주연 배우부터 5명 이름으로 수집, 절대 popularity 기준 쓰면 안 됨)

7. 공식문서: https://developer.themoviedb.org/reference/movie-credits
8. 수집 방식: 비동기 방식
9. 수집 결과: 28290 rows × 19 columns, 약 16분 소요

In [1]:
# ==========================================================
# TMDB MOVIE FULL COLLECTOR — PRODUCTION V9 (Extreme-Day FULL)
# (2005-01-01 ~ 2025-11-30)
#
# - 대상: TMDB 영화 (vote_count >= 30 + imdb_id 필수)
# - 특징:
#   * discover/movie 기간 자동 분할 (500페이지 제한 대응)
#   * ✅ 어떤 구간도 500페이지 초과 상태로 수집하지 않음
#   * ✅ 만약 "단일 날짜"에서 total_pages > 500 발생 시 → RuntimeError 발생
#       → TMDB API 한계로 그 날짜는 전수 수집 불가하다는 것을 명시적으로 알려줌
#       → 즉, "조용한 누락"은 절대 없음 (0% 누락 보장 or 실패)
#   * 감독 / 작가 / 주연배우(Top 5) 크레딧 전수 수집
#   * 초당 요청 제한(35 req/s) + 재시도(backoff)
#   * JSONL 스트림 저장 + Parquet 최종 저장
#   * 중단/재개: checkpoint + JSONL + failed_ids
# ==========================================================

import os
import json
import asyncio
import aiohttp
from collections import deque
from datetime import datetime, timedelta
from pathlib import Path
import pandas as pd
import re
from dotenv import load_dotenv
load_dotenv()

try:
    import nest_asyncio
    nest_asyncio.apply()
except Exception:
    pass

# ==========================================================
# CONFIG
# ==========================================================
DATA_DIR = Path("./tmdb_movie_full")
DATA_DIR.mkdir(exist_ok=True)

MOVIE_IDS_PATH     = DATA_DIR / "movie_ids_2005_2025.json"
DETAILS_JSONL_PATH = DATA_DIR / "movie_details_2005_2025.jsonl"
CHECKPOINT_PATH    = DATA_DIR / "checkpoint.json"
FAILED_IDS_PATH    = DATA_DIR / "failed_ids.json"
FINAL_PARQUET_PATH = DATA_DIR / "tmdb_movies_2005_2025_FULL.parquet"

API_KEY = os.getenv("TMDB_API_KEY")
if not API_KEY:
    raise ValueError("환경변수 TMDB_API_KEY 설정 필요!")

BASE_URL = "https://api.themoviedb.org/3"
HEADERS  = {"accept": "application/json"}

TIMEOUT   = 10
MAX_RETRIES = 5
MAX_CALLS_PER_SECOND   = 35
MAX_CONCURRENT_REQUESTS = 150  # 효율성

START_DATE = datetime(2005, 1, 1)
END_DATE   = datetime(2025, 11, 30)

WRITER_JOBS = {
    "Writer", "Screenplay", "Story", "Creator",
    "Novel", "Comic Book", "Adaptation", "Screenstory"
}

jsonl_lock      = asyncio.Lock()
checkpoint_lock = asyncio.Lock()

# ==========================================================
# Rate Limiter
# ==========================================================
class RateLimiter:
    def __init__(self, max_calls=35, period=1.0):
        self.max_calls = max_calls
        self.period = period
        self.calls = deque()
        self.lock  = asyncio.Lock()

    async def acquire(self):
        async with self.lock:
            loop = asyncio.get_running_loop()
            now = loop.time()

            # 오래된 호출 제거
            while self.calls and now - self.calls[0] > self.period:
                self.calls.popleft()

            if len(self.calls) >= self.max_calls:
                wait = self.period - (now - self.calls[0]) + 0.02
                if wait > 0:
                    await asyncio.sleep(wait)

                now = loop.time()
                while self.calls and now - self.calls[0] > self.period:
                    self.calls.popleft()

            self.calls.append(loop.time())


rate_limiter = RateLimiter(MAX_CALLS_PER_SECOND)

# ==========================================================
# fetch_json
# ==========================================================
async def fetch_json(session, url, params, desc="", retries=MAX_RETRIES):
    for attempt in range(1, retries + 1):
        await rate_limiter.acquire()

        try:
            async with session.get(url, params=params, timeout=TIMEOUT) as resp:
                if resp.status == 200:
                    return await resp.json()

                if resp.status == 404:
                    return None

                if resp.status in {429, 500, 502, 503, 504}:
                    if resp.status == 429:
                        retry_after = resp.headers.get("Retry-After", "2")
                        try:
                            retry_after = int(retry_after)
                        except Exception:
                            retry_after = 2
                        await asyncio.sleep(retry_after)
                    else:
                        await asyncio.sleep(attempt * 2)
                    continue

                # 기타 에러는 실패로 처리
                return None

        except asyncio.TimeoutError:
            await asyncio.sleep(attempt * 2)
        except Exception:
            await asyncio.sleep(attempt * 2)

    return None

# ==========================================================
# 문자열 정제 유틸
# ==========================================================
def sanitize_text_mild(text):
    """
    경미한 정제 (이름, 제목용)
    - 줄바꿈, HTML 태그, 연속 공백만 제거
    - 유니코드 보존 (한글, 일본어, 유럽어 등)
    """
    if not text:
        return ""
    
    text = str(text)
    
    # HTML 태그 제거
    text = re.sub(r"<[^>]+>", " ", text)
    # 줄바꿈, 탭 제거
    text = text.replace("\n", " ").replace("\r", " ").replace("\t", " ")
    # 연속 공백 제거
    text = re.sub(r"\s+", " ", text)
    
    return text.strip()


def sanitize_text_strong(text):
    """
    강력한 정제 (character용 - Parquet 안정성 우선)
    - 제어 문자, 특수 유니코드 제거
    - 일반 유니코드는 보존
    """
    if not text:
        return ""
    
    text = str(text)
    
    # HTML 태그 제거
    text = re.sub(r"<[^>]+>", " ", text)
    # HTML 엔티티 제거
    text = re.sub(r"&[a-zA-Z]+;", " ", text)
    # 제어 문자 제거
    text = re.sub(r"[\x00-\x1F\x7F-\x9F]", " ", text)
    # 연속 공백 제거
    text = re.sub(r"\s+", " ", text)
    
    return text.strip()


def join_values(arr, use_strong_sanitize=False):
    """
    리스트를 세미콜론으로 join
    - use_strong_sanitize=True: character용 (강력한 정제)
    - use_strong_sanitize=False: 이름/제목용 (경미한 정제)
    """
    if not arr:
        return ""
    
    sanitize_func = sanitize_text_strong if use_strong_sanitize else sanitize_text_mild
    cleaned = [sanitize_func(x) if x is not None else "" for x in arr]
    return "; ".join(cleaned)

# ==========================================================
# DISCOVER
# ==========================================================
async def discover_total_pages(session, start_date: datetime, end_date: datetime):
    params = {
        "api_key": API_KEY,
        "language": "en-US",
        "sort_by": "primary_release_date.asc",
        "primary_release_date.gte": start_date.strftime("%Y-%m-%d"),
        "primary_release_date.lte": end_date.strftime("%Y-%m-%d"),
        "vote_count.gte": 30,
        "include_adult": "false",
        "page": 1,
    }
    url = f"{BASE_URL}/discover/movie"
    data = await fetch_json(session, url, params)
    if not data:
        return 0
    return int(data.get("total_pages", 0))


async def split_ranges(session, start_dt: datetime, end_dt: datetime, limit: int = 500):
    """
    날짜 구간을 여러 개로 나누어
    각 구간의 discover total_pages가 limit 이하가 되도록 분할.

    - 만약 "단일 날짜(s == e)"에서조차 total_pages > limit가 나오면
      TMDB API 한계로 인해 그 날짜는 전수 수집 불가능하다는 의미이므로
      RuntimeError를 발생시켜 조용한 누락을 막음.
    """
    queue = deque([(start_dt, end_dt)])
    out = []

    while queue:
        s, e = queue.popleft()
        if s > e:
            continue

        total_pages = await discover_total_pages(session, s, e)
        if total_pages == 0:
            continue

        if total_pages <= limit:
            out.append((s, e, total_pages))
            continue

        days = (e - s).days

        # 단일 날짜인데도 500페이지 초과 → TMDB discover 한계로 전수 수집 불가
        if days <= 0:
            msg = (
                f"❌ TMDB discover 한계 초과: {s.date()} 의 total_pages={total_pages} > {limit}\n"
                f"   이 날짜에 대해 discover/movie API로는 500페이지(10,000편) 이상을 전수 수집할 수 없습니다.\n"
                f"   (primary_release_date는 날짜 단위만 지원, 시간 단위 분할 불가)\n"
                f"   → 필터 조건을 완화/변경하거나, 이 날짜를 분석 범위에서 제외해야 합니다."
            )
            raise RuntimeError(msg)

        # 날짜 구간을 반씩 나누어 total_pages가 500 이하가 되도록 재귀 분할
        mid_days = days // 2
        mid = s + timedelta(days=mid_days)

        # 왼쪽: s ~ mid
        queue.append((s, mid))
        # 오른쪽: mid+1 ~ e
        if mid < e:
            queue.append((mid + timedelta(days=1), e))

    return sorted(out, key=lambda x: x[0])


async def discover_movie_ids(session):
    print("📌 Discover 단계 시작 (movie_id 수집)")

    ranges = await split_ranges(session, START_DATE, END_DATE, limit=500)
    print(f"📅 분할된 날짜 구간: {len(ranges)}")

    all_ids = set()
    url = f"{BASE_URL}/discover/movie"

    for idx, (s, e, pages) in enumerate(ranges, start=1):
        print(f"  ▷ [{idx}/{len(ranges)}] {s.date()} ~ {e.date()} (pages={pages})")

        for p in range(1, pages + 1):
            params = {
                "api_key": API_KEY,
                "language": "en-US",
                "sort_by": "primary_release_date.asc",
                "primary_release_date.gte": s.strftime("%Y-%m-%d"),
                "primary_release_date.lte": e.strftime("%Y-%m-%d"),
                "vote_count.gte": 30,
                "include_adult": "false",
                "page": p,
            }
            data = await fetch_json(session, url, params)
            if not data:
                continue

            for item in data.get("results", []):
                if item.get("vote_count", 0) >= 30:
                    all_ids.add(item["id"])

    ids = sorted(all_ids)
    MOVIE_IDS_PATH.write_text(json.dumps(ids))
    print(f"🎉 Discover 완료: 고유 movie_id {len(ids):,}개 확보")

    return ids

# ==========================================================
# EXTRACTORS
# ==========================================================
def extract_directors(crew):
    if not crew:
        return "", "", "", ""

    seen = {}
    for c in crew:
        if c.get("job") == "Director":
            pid = c.get("id")
            if pid and pid not in seen:
                seen[pid] = c

    if not seen:
        return "", "", "", ""

    arr = sorted(seen.values(), key=lambda x: x["id"])

    names    = join_values([a.get("name") for a in arr], use_strong_sanitize=False)
    ids      = join_values([a.get("id") for a in arr], use_strong_sanitize=False)
    genders  = join_values([a.get("gender") for a in arr], use_strong_sanitize=False)
    profiles = join_values([a.get("profile_path") for a in arr], use_strong_sanitize=False)

    return names, ids, genders, profiles


def extract_writers(crew):
    """
    작가 추출:
    - job이 WRITER_JOBS에 있는 경우
    - 또는 department == "Writing" 이면서 job이 존재하는 경우
    """
    if not crew:
        return "", "", "", "", ""

    seen = {}

    for c in crew:
        job  = c.get("job")
        dept = c.get("department")
        pid  = c.get("id")

        if not pid:
            continue

        if job in WRITER_JOBS:
            if pid not in seen:
                seen[pid] = {"person": c, "jobs": set()}
            seen[pid]["jobs"].add(job)
        elif dept == "Writing" and job:
            if pid not in seen:
                seen[pid] = {"person": c, "jobs": set()}
            seen[pid]["jobs"].add(job)

    if not seen:
        return "", "", "", "", ""

    arr = sorted(seen.items(), key=lambda x: x[0])

    names    = join_values([v["person"].get("name") for _, v in arr], use_strong_sanitize=False)
    ids      = join_values([v["person"].get("id") for _, v in arr], use_strong_sanitize=False)
    roles    = join_values([
        "/".join(sorted(v["jobs"])) if v["jobs"] else "Writer"
        for _, v in arr
    ], use_strong_sanitize=False)
    genders  = join_values([v["person"].get("gender") for _, v in arr], use_strong_sanitize=False)
    profiles = join_values([v["person"].get("profile_path") for _, v in arr], use_strong_sanitize=False)

    return names, ids, roles, genders, profiles


def extract_top_cast(cast):
    """
    주연 배우 Top 5 추출
    - cast를 order 기준 오름차순 정렬
    - character가 있는 항목만 사용
    - 상위 5명까지
    - 이름은 경미 정제, character는 강력 정제
    """
    if not cast:
        return "", "", "", "", "", ""

    sorted_cast = sorted(cast, key=lambda x: x.get("order", 999))
    filtered = [c for c in sorted_cast if c.get("character")]

    top_5 = filtered[:5]
    if not top_5:
        return "", "", "", "", "", ""

    names      = join_values([c.get("name") for c in top_5], use_strong_sanitize=False)
    ids        = join_values([c.get("id") for c in top_5], use_strong_sanitize=False)
    orders     = join_values([c.get("order") for c in top_5], use_strong_sanitize=False)
    characters = join_values([c.get("character") for c in top_5], use_strong_sanitize=True)
    genders    = join_values([c.get("gender") for c in top_5], use_strong_sanitize=False)
    profiles   = join_values([c.get("profile_path") for c in top_5], use_strong_sanitize=False)

    return names, ids, orders, characters, genders, profiles

# ==========================================================
# MOVIE DETAILS FETCHER
# ==========================================================
async def fetch_movie_detail(session, movie_id: int):
    url = f"{BASE_URL}/movie/{movie_id}"
    params = {
        "api_key": API_KEY,
        "language": "en-US",
        "append_to_response": "credits",
    }

    data = await fetch_json(session, url, params, desc=f"movie/{movie_id}")
    if not data:
        return None

    imdb_id = data.get("imdb_id")
    if not imdb_id:
        return None

    vote_count = data.get("vote_count", 0)
    if vote_count < 30:
        return None

    title = sanitize_text_mild(data.get("title"))

    credits = data.get("credits", {})
    cast = credits.get("cast", []) or []
    crew = credits.get("crew", []) or []

    directors, director_ids, director_gender, director_profile = extract_directors(crew)
    writers, writer_ids, writer_roles, writer_gender, writer_profile = extract_writers(crew)
    top_cast, top_cast_ids, top_cast_order, characters, top_cast_gender, top_cast_profile = extract_top_cast(cast)

    return {
        "imdb_id": imdb_id,
        "movie_id": movie_id,
        "vote_count": vote_count,
        "title": title,

        "director_ids": director_ids,
        "directors": directors,
        "director_gender": director_gender,
        "director_profile_path": director_profile,

        "writer_ids": writer_ids,
        "writers": writers,
        "writer_roles": writer_roles,
        "writer_gender": writer_gender,
        "writer_profile_path": writer_profile,

        "top_cast_ids": top_cast_ids,
        "top_cast_order": top_cast_order,
        "top_cast": top_cast,
        "character": characters,
        "top_cast_gender": top_cast_gender,
        "top_cast_profile_path": top_cast_profile,
    }

# ==========================================================
# CHECKPOINT
# ==========================================================
def load_checkpoint():
    if CHECKPOINT_PATH.exists():
        try:
            data = json.loads(CHECKPOINT_PATH.read_text())
            return {
                "done_ids": set(data.get("done_ids", [])),
                "failed_ids": set(data.get("failed_ids", [])),
                "last_index": data.get("last_index", 0),
            }
        except Exception:
            return {"done_ids": set(), "failed_ids": set(), "last_index": 0}
    return {"done_ids": set(), "failed_ids": set(), "last_index": 0}


async def save_checkpoint(done_ids, failed_ids, last_index: int):
    async with checkpoint_lock:
        try:
            data = {
                "done_ids": list(done_ids),
                "failed_ids": list(failed_ids),
                "last_index": last_index,
                "timestamp": datetime.now().isoformat(),
            }
            tmp_path = CHECKPOINT_PATH.with_suffix(".tmp")
            tmp_path.write_text(json.dumps(data, ensure_ascii=False, indent=2))
            tmp_path.replace(CHECKPOINT_PATH)
        except Exception as e:
            print(f"⚠️ 체크포인트 저장 실패: {e}")


def load_done_from_jsonl():
    if not DETAILS_JSONL_PATH.exists():
        return set()

    done = set()
    with open(DETAILS_JSONL_PATH, "r", encoding="utf-8") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            try:
                rec = json.loads(line)
                mid = rec.get("movie_id")
                if mid:
                    done.add(mid)
            except Exception:
                pass
    return done


def save_failed_ids(failed_ids):
    FAILED_IDS_PATH.write_text(
        json.dumps(sorted(list(failed_ids)), ensure_ascii=False, indent=2)
    )

# ==========================================================
# BATCH PROCESSING
# ==========================================================
async def process_batch(session, batch, done_ids: set, failed_ids: set):
    sem = asyncio.Semaphore(MAX_CONCURRENT_REQUESTS)

    async def worker(mid: int):
        if mid in done_ids or mid in failed_ids:
            return None
        async with sem:
            result = await fetch_movie_detail(session, mid)
            if result is None:
                failed_ids.add(mid)
                return None
            return result

    tasks = [asyncio.create_task(worker(mid)) for mid in batch]
    results = await asyncio.gather(*tasks)

    valid = [r for r in results if r is not None]

    if valid:
        async with jsonl_lock:
            try:
                lines = [json.dumps(r, ensure_ascii=False) + "\n" for r in valid]
                with open(DETAILS_JSONL_PATH, "a", encoding="utf-8") as f:
                    f.writelines(lines)

                for r in valid:
                    done_ids.add(r["movie_id"])

            except Exception as e:
                print(f"⚠️ JSONL 쓰기 실패: {e}")

    return len(valid)


def chunked(arr, n):
    for i in range(0, len(arr), n):
        yield arr[i:i+n]

# ==========================================================
# MAIN
# ==========================================================
async def main():
    start = datetime.now()

    async with aiohttp.ClientSession(headers=HEADERS) as session:
        if MOVIE_IDS_PATH.exists():
            movie_ids = json.loads(MOVIE_IDS_PATH.read_text())
            print(f"📂 기존 movie_ids 로드: {len(movie_ids):,}")
        else:
            movie_ids = await discover_movie_ids(session)

        checkpoint = load_checkpoint()
        done_ids   = checkpoint["done_ids"]
        failed_ids = checkpoint["failed_ids"]

        done_ids |= load_done_from_jsonl()

        remaining = [mid for mid in movie_ids if mid not in done_ids and mid not in failed_ids]
        print(f"⏳ 처리 대상: {len(remaining):,}")
        print(f"   완료: {len(done_ids):,}, 실패: {len(failed_ids):,}")

        if not DETAILS_JSONL_PATH.exists():
            DETAILS_JSONL_PATH.touch()

        BATCH_SIZE = 400
        total_processed = 0

        batches = list(chunked(remaining, BATCH_SIZE))

        for idx, batch in enumerate(batches, start=1):
            batch_start = datetime.now()
            n = await process_batch(session, batch, done_ids, failed_ids)
            total_processed += n

            elapsed = (datetime.now() - batch_start).total_seconds()
            print(f"  ▷ Batch {idx}/{len(batches)}: {n}개 처리, 누적={total_processed}, {elapsed:.1f}s")

            if idx % 10 == 0:
                await save_checkpoint(done_ids, failed_ids, idx)
                save_failed_ids(failed_ids)
                print(f"💾 체크포인트 저장 (batch={idx})")

        await save_checkpoint(done_ids, failed_ids, len(batches))
        save_failed_ids(failed_ids)

    # JSONL → Parquet
    print("\n📦 JSONL → Parquet 변환 중...")

    if not DETAILS_JSONL_PATH.exists():
        print("⚠️ JSONL 파일이 없어 Parquet 변환을 건너뜀")
        return

    df = pd.read_json(DETAILS_JSONL_PATH, lines=True)

    df = df[df["imdb_id"].notna()]
    df = df[df["movie_id"].notna()]
    df = df.drop_duplicates(subset=["movie_id"], keep="first").reset_index(drop=True)

    cols = [
        "imdb_id", "movie_id", "vote_count", "title",
        "director_ids", "directors", "director_gender", "director_profile_path",
        "writer_ids", "writers", "writer_roles", "writer_gender", "writer_profile_path",
        "top_cast_ids", "top_cast_order", "top_cast", "character",
        "top_cast_gender", "top_cast_profile_path",
    ]

    missing = [c for c in cols if c not in df.columns]
    if missing:
        print(f"⚠️ 누락 컬럼 자동 생성: {missing}")
        for c in missing:
            df[c] = ""

    df = df[cols]

    df["movie_id"]   = df["movie_id"].astype("int32")
    df["vote_count"] = df["vote_count"].astype("int32")

    for col in df.columns:
        if col not in ["movie_id", "vote_count"]:
            df[col] = df[col].astype("string")

    df.to_parquet(FINAL_PARQUET_PATH, index=False)

    elapsed = (datetime.now() - start).total_seconds() / 60
    print(f"\n🎉 전체 파이프라인 완료! ({elapsed:.1f}분)")
    print(f"📁 저장 파일: {FINAL_PARQUET_PATH}")
    print(f"   총 영화 수: {len(df):,}편")
    print(f"   실패 영화 수: {len(json.loads(FAILED_IDS_PATH.read_text())) if FAILED_IDS_PATH.exists() else 0:,}편")


if __name__ == "__main__":
    asyncio.run(main())

📌 Discover 단계 시작 (movie_id 수집)
📅 분할된 날짜 구간: 4
  ▷ [1/4] 2005-01-01 ~ 2010-03-25 (pages=266)
  ▷ [2/4] 2010-03-26 ~ 2015-06-17 (pages=364)
  ▷ [3/4] 2015-06-18 ~ 2020-09-08 (pages=460)
  ▷ [4/4] 2020-09-09 ~ 2025-11-30 (pages=331)
🎉 Discover 완료: 고유 movie_id 28,384개 확보
⏳ 처리 대상: 23,984
   완료: 4,399, 실패: 1
  ▷ Batch 1/60: 400개 처리, 누적=400, 11.4s
  ▷ Batch 2/60: 399개 처리, 누적=799, 11.4s
  ▷ Batch 3/60: 400개 처리, 누적=1199, 12.1s
  ▷ Batch 4/60: 400개 처리, 누적=1599, 11.8s
  ▷ Batch 5/60: 400개 처리, 누적=1999, 12.1s
  ▷ Batch 6/60: 400개 처리, 누적=2399, 12.0s
  ▷ Batch 7/60: 399개 처리, 누적=2798, 12.0s
  ▷ Batch 8/60: 399개 처리, 누적=3197, 12.0s
  ▷ Batch 9/60: 398개 처리, 누적=3595, 12.0s
  ▷ Batch 10/60: 398개 처리, 누적=3993, 12.1s
💾 체크포인트 저장 (batch=10)
  ▷ Batch 11/60: 400개 처리, 누적=4393, 12.1s
  ▷ Batch 12/60: 400개 처리, 누적=4793, 11.9s
  ▷ Batch 13/60: 400개 처리, 누적=5193, 11.9s
  ▷ Batch 14/60: 400개 처리, 누적=5593, 11.9s
  ▷ Batch 15/60: 400개 처리, 누적=5993, 11.8s
  ▷ Batch 16/60: 399개 처리, 누적=6392, 11.8s
  ▷ Batch 17/60: 400개 처리, 누적=